##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# DeepDream

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/generative/deepdream"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/deepdream.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/deepdream.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/generative/deepdream.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

Это руководство содержит минимальную реализацию DeepDream, как описано в этом [сообщении в блоге](https://ai.googleblog.com/2015/06/inceptionism-going-deeper-into-neural.html) Александра Мордвинцева.

DeepDream - это эксперимент, который визуализирует закономерности, полученные нейронной сетью. Подобно тому, как ребенок наблюдает за облаками и пытается интерпретировать случайные формы, DeepDream чрезмерно интерпретирует и усиливает узоры, которые он видит на изображении.

Это делается путем пересылки изображения по нейронной сети с последующим вычислением градиента изображения по отношению к активациям определенного слоя. Затем изображение модифицируется, чтобы усилить эти активации, усилить паттерны, видимые сетью, и в результате получить образ, похожий на сон. Этот процесс получил название «Inceptionism»(ссылка на [InceptionNet](https://arxiv.org/pdf/1409.4842.pdf) и [фильм Начало](https://en.wikipedia.org/wiki/Inception)).

Давайте посмотрим, как можно сделать нейронную сеть «мечтой» и улучшить сюрреалистические паттерны, которые она видит на изображении.

![Dogception](https://www.tensorflow.org/tutorials/generative/images/dogception.png)

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np

import matplotlib as mpl

import IPython.display as display
import PIL.Image

from tensorflow.keras.preprocessing import image

## Выбор изображения для визуализации "сна"

В этом уроке давайте будем использовать изображение [лабрадора](https://commons.wikimedia.org/wiki/File:YellowLabradorLooking_new.jpg).

In [ ]:
url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg'

In [ ]:
# Загружаем изображение и читаем его в NumPy array.
def download(url, max_dim=None):
  name = url.split('/')[-1]
  image_path = tf.keras.utils.get_file(name, origin=url)
  img = PIL.Image.open(image_path)
  if max_dim:
    img.thumbnail((max_dim, max_dim))
  return np.array(img)

# Нормализуем изображение
def deprocess(img):
  img = 255*(img + 1.0)/2.0
  return tf.cast(img, tf.uint8)

# Выводим изображение
def show(img):
  display.display(PIL.Image.fromarray(np.array(img)))


# Уменьшаем размер изображения для упрощения работы с ним
original_img = download(url, max_dim=500)
show(original_img)
display.display(display.HTML('Image cc-by: <a "href=https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg">Von.grzanka</a>'))

## Подготовка модели извлечения признаков

Загрузите и подготовьте предварительно обученную модель классификации изображений. Вы будете использовать [InceptionV3](https://keras.io/applications/#inceptionv3), которая похожа на модель, изначально использовавшуюся в DeepDream. Обратите внимание, что любая [предварительно обученная модель](https://keras.io/applications/#models-for-image-classification-with-weights-trained-on-imagenet) будет здесь работать, однако вам придется настроить имена слоев.

In [ ]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

Идея DeepDream состоит в том, чтобы выбрать слой(или слои) и максимизировать «потери» таким образом, чтобы изображение все больше «возбуждало» слои. Сложность включенных функций зависит от выбранных вами слоев, то есть нижние слои создают штрихи или простые узоры, а более глубокие слои придают сложные функции изображениям или даже целым объектам.

Архитектура InceptionV3 довольно большая(диаграмму архитектуры модели см. в [исследовательском репозитории TensorFlow](https://github.com/tensorflow/models/tree/master/research/inception)). Для DeepDream нас интересуют те слои, в которых свертки объединены. В InceptionV3 есть 11 таких слоев, которые имеют имена от «mixed0» до «mixed10». Использование разных слоев приведет к получению разных сказочных изображений. Более глубокие слои реагируют на элементы более высокого уровня (например, глаза и лица), тогда как более ранние слои реагируют на более простые элементы (такие как края, формы и текстуры). Не стесняйтесь экспериментировать со слоями, но имейте в виду, что более глубокие слои(с более высоким индексом) потребуют больше времени для обучения, поскольку вычисление градиента более глубокое.

In [ ]:
# максимизируем активацию этих слоев
names = ['mixed3', 'mixed5']
layers = [base_model.get_layer(name).output for name in names]

# создаем модель извлечения признаков
dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)

## Расчет величины потерь

Потери - это сумма активаций в выбранных слоях. Потери нормализованы на каждом слое, поэтому вклад более крупных слоев не перевешивает меньшие слои. Обычно потери - это величина, которую вы хотите минимизировать с помощью градиентного спуска. В DeepDream вы максимизируете эту потерю с помощью градиентного подъема.

In [ ]:
def calc_loss(img, model):
  # Передаем изображение через модель, чтобы получить активации.
  # Преобразуем изображение в пакет размером 1.
  img_batch = tf.expand_dims(img, axis=0)
  layer_activations = model(img_batch)
  if len(layer_activations) == 1:
    layer_activations = [layer_activations]

  losses = []
  for act in layer_activations:
    loss = tf.math.reduce_mean(act)
    losses.append(loss)

  return  tf.reduce_sum(losses)

## Градиентный подъем

После того, как вы рассчитали потери для выбранных слоев, осталось вычислить градиенты по отношению к изображению и добавить их к исходному изображению.

Добавление градиентов к изображению усиливает узоры, видимые сетью. На каждом этапе вы будете создавать изображение, которое все больше и больше возбуждает активацию определенных слоев в сети.

Метод, который делает это,  заключен в `tf.function`, для повышения производительности. Он использует `input_signature`, чтобы гарантировать, что функция не будет восстанавливаться для разных размеров изображения или значений `steps`/`step_size`. Подробности см. в [Руководство по конкретным функциям](../../guide/concrete_function.ipynb).

In [ ]:
class DeepDream(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(
      input_signature=(
        tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
        tf.TensorSpec(shape=[], dtype=tf.int32),
        tf.TensorSpec(shape=[], dtype=tf.float32),)
  )
  def __call__(self, img, steps, step_size):
      print("Tracing")
      loss = tf.constant(0.0)
      for n in tf.range(steps):
        with tf.GradientTape() as tape:
          # Для этого нужны градиенты относительно `img`
          # По умолчанию `GradientTape` следит только за `tf.Variable`s
          tape.watch(img)
          loss = calc_loss(img, self.model)

        # Вычисляем градиент потерь по отношению к пикселям входного изображения.
        gradients = tape.gradient(loss, img)

        # Нормализуем градиенты
        gradients /= tf.math.reduce_std(gradients) + 1e-8 
        
        # При градиентном подъеме «потери» максимизируются, 
        # так что входное изображение все больше «возбуждает» слои.
        # Вы можете обновить изображение, напрямую добавив градиенты(потому что они одинаковой размерности!)
        img = img + gradients*step_size
        img = tf.clip_by_value(img, -1, 1)

      return loss, img

In [ ]:
deepdream = DeepDream(dream_model)

## Основной цикл

In [ ]:
def run_deep_dream_simple(img, steps=100, step_size=0.01):
  # Преобразовываем из uint8 в диапазон, ожидаемый моделью.
  img = tf.keras.applications.inception_v3.preprocess_input(img)
  img = tf.convert_to_tensor(img)
  step_size = tf.convert_to_tensor(step_size)
  steps_remaining = steps
  step = 0
  while steps_remaining:
    if steps_remaining>100:
      run_steps = tf.constant(100)
    else:
      run_steps = tf.constant(steps_remaining)
    steps_remaining -= run_steps
    step += run_steps

    loss, img = deepdream(img, run_steps, tf.constant(step_size))
    
    display.clear_output(wait=True)
    show(deprocess(img))
    print ("Step {}, loss {}".format(step, loss))


  result = deprocess(img)
  display.clear_output(wait=True)
  show(result)

  return result

In [ ]:
dream_img = run_deep_dream_simple(img=original_img, 
                                  steps=100, step_size=0.01)

## Повышение октавы

Довольно неплохо, но с этой первой попыткой есть несколько проблем:

   1. Выходной сигнал зашумлен(это можно исправить с помощью функции потерь `tf.image.total_variation`).
   1. Изображение низкого разрешения.
   1. Создается впечатление, что все шаблоны создаются с одинаковой степенью детализации.
  
Один из подходов, который решает все эти проблемы, - это применение градиентного подъема на разных уровнях. Это позволит объединить шаблоны, созданные в меньших масштабах, в шаблоны более высоких масштабов и заполнить их дополнительными деталями.

Для этого вы можете выполнить предыдущий подход градиентного подъема, затем увеличить размер изображения(который называется октавой) и повторить этот процесс для нескольких октав.


In [ ]:
import time
start = time.time()

OCTAVE_SCALE = 1.30

img = tf.constant(np.array(original_img))
base_shape = tf.shape(img)[:-1]
float_base_shape = tf.cast(base_shape, tf.float32)

for n in range(-2, 3):
  new_shape = tf.cast(float_base_shape*(OCTAVE_SCALE**n), tf.int32)

  img = tf.image.resize(img, new_shape).numpy()

  img = run_deep_dream_simple(img=img, steps=50, step_size=0.01)

display.clear_output(wait=True)
img = tf.image.resize(img, base_shape)
img = tf.image.convert_image_dtype(img/255.0, dtype=tf.uint8)
show(img)

end = time.time()
end-start

## Опционально: масштабирование с помощью фрагментов

Следует учитывать, что по мере увеличения размера изображения увеличиваются время и память, необходимые для вычисления градиента. Вышеупомянутая реализация повышения октав не будет работать с очень большими изображениями или с большим количеством октав.

Чтобы избежать этой проблемы, вы можете разделить изображение на фрагменты и вычислить градиент для каждого фрагмента.

Применение случайных сдвигов к изображению перед каждым вычислением мозаики предотвращает появление стыков фрагментов.

Начнем с реализации случайного сдвига:

In [ ]:
def random_roll(img, maxroll):
  # Произвольно сдвигаем изображение, чтобы избежать мозаичных стыков.
  shift = tf.random.uniform(shape=[2], minval=-maxroll, maxval=maxroll, dtype=tf.int32)
  shift_down, shift_right = shift[0],shift[1] 
  img_rolled = tf.roll(tf.roll(img, shift_right, axis=1), shift_down, axis=0)
  return shift_down, shift_right, img_rolled

In [ ]:
shift_down, shift_right, img_rolled = random_roll(np.array(original_img), 512)
show(img_rolled)

Вот мозаичный эквивалент функции `deepdream`, определенной ранее:

In [ ]:
class TiledGradients(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(
      input_signature=(
        tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
        tf.TensorSpec(shape=[], dtype=tf.int32),)
  )
  def __call__(self, img, tile_size=512):
    shift_down, shift_right, img_rolled = random_roll(img, tile_size)

    # Устанавливаем нулевые градиенты изображения.
    gradients = tf.zeros_like(img_rolled)
    
    # Пропускаем последний фрагмент, если только он не один.
    xs = tf.range(0, img_rolled.shape[0], tile_size)[:-1]
    if not tf.cast(len(xs), bool):
      xs = tf.constant([0])
    ys = tf.range(0, img_rolled.shape[1], tile_size)[:-1]
    if not tf.cast(len(ys), bool):
      ys = tf.constant([0])

    for x in xs:
      for y in ys:
        # Расчитываем градиент для фрагмента
        with tf.GradientTape() as tape:
          # Для этого нужны градиенты относительно `img_rolled`.
          # По умолчанию `GradientTape` отслеживает только `tf.Variable`.
          tape.watch(img_rolled)

          # Извлекаем фрагмент из изображения.
          img_tile = img_rolled[x:x+tile_size, y:y+tile_size]
          loss = calc_loss(img_tile, self.model)

        # Обновляем градиент изображения для этого фрагмента
        gradients = gradients + tape.gradient(loss, img_rolled)

    # Отменяем случайный сдвиг, примененный к изображению и его градиентам.
    gradients = tf.roll(tf.roll(gradients, -shift_right, axis=1), -shift_down, axis=0)

    # Нормализуем градиенты
    gradients /= tf.math.reduce_std(gradients) + 1e-8 

    return gradients 

In [ ]:
get_tiled_gradients = TiledGradients(dream_model)

Объединение всего этого дает масштабируемую реализацию deepdream с учетом октавы:

In [ ]:
def run_deep_dream_with_octaves(img, steps_per_octave=100, step_size=0.01, 
                                octaves=range(-2,3), octave_scale=1.3):
  base_shape = tf.shape(img)
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = tf.keras.applications.inception_v3.preprocess_input(img)

  initial_shape = img.shape[:-1]
  img = tf.image.resize(img, initial_shape)
  for octave in octaves:
    # Масштабируем изображение по октаве
    new_size = tf.cast(tf.convert_to_tensor(base_shape[:-1]), tf.float32)*(octave_scale**octave)
    img = tf.image.resize(img, tf.cast(new_size, tf.int32))

    for step in range(steps_per_octave):
      gradients = get_tiled_gradients(img)
      img = img + gradients*step_size
      img = tf.clip_by_value(img, -1, 1)

      if step % 10 == 0:
        display.clear_output(wait=True)
        show(deprocess(img))
        print ("Octave {}, Step {}".format(octave, step))
    
  result = deprocess(img)
  return result

In [ ]:
img = run_deep_dream_with_octaves(img=original_img, step_size=0.01)

display.clear_output(wait=True)
img = tf.image.resize(img, base_shape)
img = tf.image.convert_image_dtype(img/255.0, dtype=tf.uint8)
show(img)

Намного лучше! Поиграйтесь с количеством октав, октавной шкалой и активированными слоями, чтобы изменить внешний вид вашего изображения в DeepDream.

Вас также может заинтересовать [TensorFlow Lucid](https://github.com/tensorflow/lucid), который расширяет идеи, представленные в этом руководстве, для визуализации и интерпретации нейронных сетей.